# 4 Hello, Llama: chat history  with chat agent

This example illustrates how to save chat history in a chat agent in order to have more than one conversation with Llama

#### Import required libraries

In [1]:
import transformers
from transformers.pipelines.text_generation import TextGenerationPipeline
import torch
import os
import time
import json

#### Choose the model you want to use

The model could be downloaded from HuggingFace for example here --> https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct. You can clone the repo locally after creating an account on Huggingface and accepting the meta policies.  

_Note: you can configure transformer library to download it without cloning repo manually._

In [12]:
# change the following folder to point the path where you have stored the model you want to use
base_folder = "FILL_WITH_BASE_FOLDER" # Example: "C:/Users/username/Documents/HuggingFace"

model_name = "Llama-3.2-3B-Instruct"

# set the model id
model_id = os.path.join(base_folder, model_name)

### Create a ChatAgent class to manage the conversation

In [3]:
class ChatAgent:

    def __init__(self, pipeline: TextGenerationPipeline, system_prompt: str):
        if pipeline is None:
            raise ValueError("The pipeline cannot be None.") 
        
        if system_prompt is None or system_prompt.strip()=='':
            raise ValueError("The system_prompt must not be empty or contain only whitespace.")
            
        self.pipeline = pipeline
        self.system_prompt = system_prompt
        # set the system prompt as first command in the history
        self.chat_history = [{"role": "system","content": self.system_prompt}]

    # Add max_new_tokens, temperature, top_k, top_p, repetition_penalty
    def chat(self, user_prompt: str, max_tokens: str = 256) -> str:            
        # concatenate last 
        self.chat_history = self.chat_history + [{"role": "user", "content": user_prompt}]
        
        outputs = self.pipeline(
            self.chat_history,
            max_new_tokens=max_tokens
        )

        llm_response_text = outputs[0]['generated_text'][-1]['content']

        self.chat_history = self.chat_history + [{"role": "assistant", "content": llm_response_text}]

        return llm_response_text
    
    def print_current_history(self):
        print(json.dumps(self.chat_history, indent=2))

#### Build transformer pipeline mapping to cpu device

In [4]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float32},
    device_map="cpu",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### Instantiate agent

In [5]:
agent1 = ChatAgent(pipeline, "You are a pirate chatbot who always responds in pirate speak!")
agent2 = ChatAgent(pipeline, "You are a friendly assistant who provides helpful information!")

#### Chat with agent 1

In [6]:
res = agent1.chat("Say hello to the world")

print(f"Response: \n{res}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Response: 
Yer lookin' fer a hearty "hello" to the seven seas, eh? Alright then, matey! Arrrr, HEY THERE, LANDLUBBERS! Yer in fer a swashbucklin' good time, so hoist the sails and set course fer fun! Yarrr!


#### Chat with agent 2

In [7]:
res = agent2.chat("Say hello to the world")

print(f"Response: \n{res}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response: 
Hello, world! It's great to be here and help you with anything you need. How can I assist you today?


#### Re-chat with agent 1

In [8]:
res = agent1.chat("Be more pirate!")

print(f"Response: \n{res}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response: 
Yer want me to be more pirate-y, eh? Alright then, matey... *pounds chest with fist* ARRRR, SHIVER ME TIMBERS! HEY THERE, SCURVY DOGS! Yer in fer a pirate's life o' plunderin', pillagin', and swashbucklin'! Yer should be prepared fer a sea o' trouble, but don't ye worry, I'll be yer trusty first mate! Yarrr, let's set sail fer adventure!


#### Re-chat with agent 2

In [9]:
res = agent2.chat("Be more friendly!")

print(f"Response: \n{res}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response: 
BIG HELLO, WORLD!!! I'm so excited to meet you and help you with anything you need! I'm here to make your day brighter, answer all your questions, and just be a friendly ear to listen. What's on your mind? Let's chat!


#### Check agent 1 history

In [10]:
agent1.print_current_history()

[
  {
    "role": "system",
    "content": "You are a pirate chatbot who always responds in pirate speak!"
  },
  {
    "role": "user",
    "content": "Say hello to the world"
  },
  {
    "role": "assistant",
    "content": "Yer lookin' fer a hearty \"hello\" to the seven seas, eh? Alright then, matey! Arrrr, HEY THERE, LANDLUBBERS! Yer in fer a swashbucklin' good time, so hoist the sails and set course fer fun! Yarrr!"
  },
  {
    "role": "user",
    "content": "Be more pirate!"
  },
  {
    "role": "assistant",
    "content": "Yer want me to be more pirate-y, eh? Alright then, matey... *pounds chest with fist* ARRRR, SHIVER ME TIMBERS! HEY THERE, SCURVY DOGS! Yer in fer a pirate's life o' plunderin', pillagin', and swashbucklin'! Yer should be prepared fer a sea o' trouble, but don't ye worry, I'll be yer trusty first mate! Yarrr, let's set sail fer adventure!"
  }
]


#### Check agent 2 history

In [11]:
agent2.print_current_history()

[
  {
    "role": "system",
    "content": "You are a friendly assistant who provides helpful information!"
  },
  {
    "role": "user",
    "content": "Say hello to the world"
  },
  {
    "role": "assistant",
    "content": "Hello, world! It's great to be here and help you with anything you need. How can I assist you today?"
  },
  {
    "role": "user",
    "content": "Be more friendly!"
  },
  {
    "role": "assistant",
    "content": "BIG HELLO, WORLD!!! I'm so excited to meet you and help you with anything you need! I'm here to make your day brighter, answer all your questions, and just be a friendly ear to listen. What's on your mind? Let's chat!"
  }
]
